# 1 Patient Selection
Summary:
1. In this script we elucidate the first consult for each patient in HIX
2. We remove patients that were admitted before 2011 (because we cannot guarantee that we have all the data avail)
3. We export / save different patient selections to be used in the proceeding research

In [1]:

PATH_SCRIPTS= r'../src/scripts'
PATH_MODULES= r'/exports/reum/tdmaarseveen/modules/' # unused
PATH_FILES = r"../../EMR_mining/output_files/"

import sys
sys.path.append(PATH_SCRIPTS)
sys.path.append(r'../src/1_emr_scripts')
#sys.path.append(PATH_MODULES)

import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Preload function
def getFirstPEC(pat):
    if pat in clean_dict.keys():
        return clean_dict[int(pat)]
    else :
        return np.nan

## #1 Import DDR_A
We will leverage the conclusion section of the DDR_A table (HIX) for our prediction of the RA diagnosis.

### #1.1 Import from latest HIX extraction (PUB)
- keep in mind patient ids are pseudonymized in this version
- We chose to not use the latest HiX patients because we did not have ethical consent for these patients

In [2]:
import pandas as pd

df_REUTOT = pd.read_csv(r'../data/1_raw/Clustering_Conclusion_Consults.csv', sep=";", header = None)
df_REUTOT.columns = ['id', 'STELLING', 'XANTWOORD', 'DATUM', 'PATNR', 'DATUM_A', 'Author']

# Key file to link pseudonymized keys to pat ids
df_pat = pd.read_csv(r'../data/1_raw/Pseudonymized_Keys.csv', sep=';')

# Create dictionary from key file
di = dict(zip(df_pat.member_entity_Pseudonym_value, df_pat.member_entity_Patient_value))
def unblind(x):
    try:
        return di[x]
    except:
        return 0

# Unblind
df_REUTOT['PATNR'] = df_REUTOT['PATNR'].astype(int).apply(lambda x: unblind(x))

/home/tdmaarseveen/.conda/envs/ra_clustering2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### #1.2 Import data freeze (MI_uitgifte)

In [3]:
import pandas as pd
df_REUTOT = pd.read_csv(PATH_FILES + r'DF_REU_tot.csv', sep="|")
df_REUTOT['DATUM_A'] = pd.to_datetime(df_REUTOT['DATUM_A'], format='%Y-%m-%d', errors='ignore') # DATUM_A is most reliable
df_REUTOT['DATUM'] = pd.to_datetime(df_REUTOT['DATUM'], format='%Y-%m-%d', errors='ignore') # DATUM_A is most reliable
len(df_REUTOT[df_REUTOT['STELLING']=='Type contact/bespreking']['PATNR'].unique())

/home/tdmaarseveen/.conda/envs/ra_clustering2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


25993

In [105]:
df_pat = pd.read_csv(r'../filters/RA_patients_083_new.csv', sep=',', index_col=0)
new_pat = dict(zip(df_pat.PATNR, df_pat.NEW_PEC))
print(len(new_pat.keys()))

1251


## #2 Acquire first consult (PEC)

First, we select patients where first consult is registered (either PVC or PEC). The PEC has to be after 29 of August 2011, otherwise we can't guarantee that we have a complete set of records (HIX became obligatory in 2011) 

Next, we impute the PEC if we can't find a first consult :
- We postulate that consults that aren't preceded by other consults in over a year most likely concern a first consult (PEC)!
- Consequently we had to employ a cut-off before imputing the PEC. To ensure that we have access to the consults in the preceding year we needed to guarantee that the information was registered in HiX. We applied a threshold, only requesting patients after August 2012 (since HiX was only initialized a mere year ago: August 2011)
- Furthermore, we removed the entries after 16-07-2020 (because this data freeze has entries up to 06-04-2019)


In [4]:
from math import isnan
import numpy as np

l_patients = {}

# create subset with consult types
sub_df = df_REUTOT[df_REUTOT['STELLING']=='Type contact/bespreking'].copy() # can we link first date to consult type?
print(len(sub_df['PATNR'].unique()))

sub_df2 = sub_df[sub_df['XANTWOORD'].isin(['pec', 'pvc'])].copy() # , 'pvc'
sub_df2 = sub_df2.sort_values('DATUM').drop_duplicates('PATNR',keep='first') # keep 'first' first consult
d_pec =sub_df2.groupby(['PATNR'])['DATUM'].agg('min').to_dict()

## Impute PEC
d_first_date =sub_df.groupby(['PATNR'])['DATUM'].agg('min').to_dict()


for pat in d_first_date.keys():
    if pat in d_pec.keys(): # IF PEC
        if d_pec[pat].tz_localize(None)==pd.to_datetime('19000101', format='%Y%m%d', errors='ignore'):
            
            d_first_date[pat] = sub_df2[pd.DatetimeIndex(sub_df2['DATUM']).tz_localize(None)==pd.to_datetime('19000101', format='%Y%m%d', errors='ignore')]['DATUM']
            print('yeah', d_first_date[pat], 'old', d_pec[pat])
        else :
            d_first_date[pat] = d_pec[pat]
        if d_pec[pat].tz_localize(None) > pd.to_datetime('20180617', format='%Y%m%d', errors='ignore'):
            d_first_date[pat] = np.nan
        # First allowed data is 29-07-2011
        if d_pec[pat].tz_localize(None) < pd.to_datetime('20110729', format='%Y%m%d', errors='ignore'): # smaller than init HIX
            d_first_date[pat] = np.nan
    else : # IF ANYTHING BUT PEC -> TRY TO IMPUTE
        date = d_first_date[pat]
        consult = sub_df[((sub_df['PATNR']==pat) & (pd.to_datetime(sub_df['DATUM'], format='%Y%m%d', errors='ignore')==date))]['XANTWOORD'].iloc[0]
        if consult != 'PEC':
            # we know that it is the first consult -> hence check if it is one year after creation of HIX
            if  date.tz_localize(None) < pd.to_datetime('20120729', format='%Y%m%d', errors='ignore'):
                d_first_date[pat] = np.nan
        # Latest allowed data is 17-06-2018
        if date.tz_localize(None) > pd.to_datetime('20180617', format='%Y%m%d', errors='ignore'):
            d_first_date[pat] = np.nan
        

# Filter out the patients without a (imputed) PEC 
clean_dict = {k: d_first_date[k] for k in d_first_date if type(d_first_date[k])!=float}

25993
yeah 10758   1900-01-01 00:00:00+00:00
10757   1900-01-01 00:00:00+00:00
10756   1900-01-01 00:00:00+00:00
Name: DATUM, dtype: datetime64[ns, UTC] old 1900-01-01 00:00:00+00:00
yeah 10758   1900-01-01 00:00:00+00:00
10757   1900-01-01 00:00:00+00:00
10756   1900-01-01 00:00:00+00:00
Name: DATUM, dtype: datetime64[ns, UTC] old 1900-01-01 00:00:00+00:00
yeah 10758   1900-01-01 00:00:00+00:00
10757   1900-01-01 00:00:00+00:00
10756   1900-01-01 00:00:00+00:00
Name: DATUM, dtype: datetime64[ns, UTC] old 1900-01-01 00:00:00+00:00


### #2.1 Export patient with first date to csv

In [5]:
cnt =0
with open('../filters/RA_imputed_NEW.dict', 'w') as f: 
    f.write('{')
    for key, value in clean_dict.items():
        #if value in ['pvc', 'pec']: # later maybe also include the possibilities aside from pvc??
        if cnt != len(clean_dict)-1:
            f.write('%s : "%s",' % (key, clean_dict[key]))
        else :
            f.write('%s : "%s"' % (key, clean_dict[key]))
        cnt += 1
    f.write('}')
cnt

23180

## #3 Apply machine learning model to identify RA patients
- select conclusion section
- format date
- assess date of first consult
- apply cut-off (only include entries after 2011 (initialization of HIX))

In [7]:
# Preload capture function that converts the first consult date from type string to type date
def capture(row):
    x = row['PEC']
    try:
        return pd.to_datetime(str(x))
    except:
        if 10757 in x: # weird artefact???
            return pd.to_datetime('2017-12-06 00:00:00+00:00', format='%Y-%m-%d', errors='ignore')
        print(row, 'Not Captured')


In [8]:
import Preprocessing as func

df_REUTOT = df_REUTOT[df_REUTOT['STELLING']=='Conclusie']
print(len(df_REUTOT))
df_REUTOT['PEC'] = df_REUTOT['PATNR'].apply(lambda x: getFirstPEC(x)) # apply first date
df_REUTOT['XANTWOORD'] = df_REUTOT['XANTWOORD'].apply(lambda x : func.processArtefactsXML(str(x)))
df_REUTOT = df_REUTOT[(~df_REUTOT['PEC'].isna())]
df_REUTOT['PEC'] = df_REUTOT.apply(lambda x: capture(x), axis=1)
df_REUTOT['DATUM'] =df_REUTOT['DATUM'].apply(lambda x: pd.to_datetime(x, utc=True, format='%Y-%m-%d', errors='ignore'))
df_REUCON = df_REUTOT[((pd.DatetimeIndex(df_REUTOT['DATUM']).tz_localize(None) < pd.DatetimeIndex(df_REUTOT['PEC']).tz_localize(None) + pd.DateOffset(years=1, months=0, days=1)) & (pd.DatetimeIndex(df_REUTOT['DATUM']).tz_localize(None) >= pd.DatetimeIndex(df_REUTOT['PEC']).tz_localize(None)))]
df_REUCON = df_REUCON[((pd.DatetimeIndex(df_REUCON['DATUM']).tz_localize(None) > pd.to_datetime('20110829', format='%Y%m%d', errors='ignore')) & (pd.DatetimeIndex(df_REUCON['PEC']).tz_localize(None) > pd.to_datetime('20110829', format='%Y%m%d', errors='ignore')))]
print(len(df_REUCON))


137466
43038


### #3.1 Preprocessing pipeline

- Cast entry based table to a patient based format.
- Perform lemmatization & remove textual artefacts

In [ ]:
import NLP_functions as func

import pickle
import time

#model_name='savedModels/SVM.sav'
X_column='XANTWOORD'
y_column=""#'Outcome'
id_column='PATNR'


def entriesPatientMerge(pat_df, id_column, X_column, y_column=""):
    """
    Merges the entries into one entry per patient (according to the id_column)
    
    Input: 
        id_column = column with patient id
        X_column = column with 
    """
    field = ''
    for i in pat_df[X_column]:
        field += " " + i + " "
    if y_column!="":
        return {X_column: field, id_column : pat_df[id_column].iloc[0], y_column : pat_df[y_column].iloc[0]}
    else :
        return {X_column: field, id_column : pat_df[id_column].iloc[0]}

if y_column != "":
    df_ult = pd.DataFrame(columns=[X_column,  id_column, y_column])
else : 
    df_ult = pd.DataFrame(columns=[X_column,  id_column])

for pat in df_REUCON[id_column].unique():
    print(pat)
    pat_df = df_REUCON[df_REUCON[id_column]==pat]
    if y_column != "":
        df_ult = df_ult.append(entriesPatientMerge(pat_df, id_column, X_column, y_column), ignore_index=True)
    else : 
        df_ult = df_ult.append(entriesPatientMerge(pat_df, id_column, X_column), ignore_index=True)

df_ult['XANTWOORD'] = df_ult['XANTWOORD'].apply(lambda x : func.lemmatizingText(x, lan='nl')) # 'en' for english & 'de' for german
df_ult=df_ult.fillna('')
df_ult['XANTWOORD'] = df_ult['XANTWOORD'].apply(lambda x : func.processArtefactsXML(str(x)))
df_ult.to_csv(r'../data/1_raw/processed_conclusions_date_2.csv', sep='|')

### #3.2 Apply SVM
- import pickled machine learning model
- write predictions to csv file

In [62]:
import pickle
import time
import sklearn
# get EMR text
text_column = 'XANTWOORD'
X = df_ult[text_column].values

# apply built model on provided text
loaded_model = pickle.load(open(r'../models/SVM_29042020.sav', 'rb')) # SVM_29042020.sav #  SVM_22012020.sav

t0 = time.time()

probas_ = loaded_model.predict_proba(X)
pred = probas_[:,1]

t1 = time.time()
print('Time for Making predictions (n=' + str(len(df_ult['PATNR'].unique())) + ') : ' + str(t1-t0))

# add predictions to table
df_ult['prediction'] = df_ult[text_column].copy()
df_ult['prediction'] = pred

df_ult[['PATNR', 'prediction']].to_csv(r'../data/1_raw/RA_patients_pred_3.csv', sep='|', index=False)

Time for Making predictions (n=17315) : 9.187673807144165


## #4 Export patient consult information

Write down important patient info in a single table:
1. Patient id
2. Probability
3. Date of first consult (or imputed)

In [67]:
# load predictions
df_ult = pd.read_csv(r'../data/1_raw/RA_patients_pred_3.csv', sep='|')

# Add first consult date
df_ult['PEC'] = df_ult['PATNR'].apply(lambda x: getFirstPEC(x))

## Export 
df_ult.to_csv('../data/1_raw/RA_patients_pred_pec.csv')

### #4.1 Update PEC 16-07-2021
- we discovered that the PEC was sometimes registered after the PVC (follow-up consult), thus we updated the function to count PVC as PEC if it was mentioned first. We updated our consult information accordingly.

In [100]:
df_pat = pd.read_csv('../data/1_raw/RA_patients_pred_pec.csv',sep=',')
df_pat['NEW_PEC'] = df_pat['PATNR'].apply(lambda x: getFirstPEC(x))
subset_df = df_pat[((df_pat['prediction']>=0.83) & (~df_pat['PEC'].isna()))].copy()
subset_df = subset_df[(pd.DatetimeIndex(pd.to_datetime(subset_df['PEC'], format='%Y%m%d', errors='ignore')).tz_localize(None) > pd.to_datetime('20110829', format='%Y%m%d', errors='ignore'))]
subset_df[['PATNR', 'prediction', 'PEC', 'NEW_PEC']].to_csv('../filters/RA_patients_083_new.csv')

In [106]:
subset_df[['prediction', 'PEC', 'NEW_PEC']]

,prediction,PEC,NEW_PEC
0,1.000000,2012-01-31 00:00:00,2012-01-31 00:00:00+00:00
3,1.000000,2017-04-03 00:00:00,2011-06-22 00:00:00+00:00
7,1.000000,2012-03-26 00:00:00,2012-03-26 00:00:00+00:00
9,1.000000,2012-01-11 00:00:00,2012-01-11 00:00:00+00:00
10,0.973550,2016-02-19 00:00:00,2012-04-10 00:00:00+00:00
...,...,...,...
22424,0.995983,2018-03-06 00:00:00,2018-03-06 00:00:00+00:00
22471,0.997036,2017-11-21 00:00:00,2017-11-21 00:00:00+00:00
22571,0.990080,2018-04-04 00:00:00,2018-04-04 00:00:00+00:00
22648,0.988933,2018-04-06 00:00:00,2018-04-06 00:00:00+00:00
